# Building an AI Agent with Gemini and GitHub API

This guide will walk you through creating a simple but powerful AI "Agent" in a Google Colab notebook. This agent will use **Google's Gemini model** to understand a user's request, use a "tool" to fetch live data from the GitHub API, and then provide a natural language summary.

### **Core Concepts you should know**

*   **What is an API? (Application Programming Interface)**
    An API is like a restaurant menu. It provides a list of pre-defined requests that one program can make to another. In our case, we will use the GitHub API to "order" information about a user. We don't need to know how GitHub's database works; we just need to use the menu (the API) correctly.

*   **What is an LLM? (Large Language Model)**
    An LLM, like Gemini, is a massive neural network trained on a vast amount of text and code. It excels at understanding, generating, and summarizing language. However, its knowledge is "frozen" at the time it was trained; it doesn't know about real-time events or private data.

*   **What is an AI Agent?**
    An AI Agent is a system that combines the language power of an LLM with the ability to take actions. It works in a loop:
    1.  **Reason:** The LLM analyzes a request and determines if it needs more information.
    2.  **Act:** If needed, it decides to use a "tool" (like calling an API).
    3.  **Observe:** It receives the result from the tool.
    4.  **Respond:** It uses the new information to formulate a final, comprehensive answer.

*   **What is Tool Calling?**
    Tool Calling is the mechanism that allows the LLM to take action or do something. The LLM doesn't run the code itself. Instead, it says, "To answer this, I need you to run the `get_github_user_info` function with the username 'google'." Our code then executes the function and passes the result back to the LLM.


## Step 1: Setting Up the Environment

Before we can write our agent, we need to install the necessary Python libraries. A library is a collection of pre-written code that helps us perform common tasks without having to write everything from scratch. We will use `pip`, the standard package installer for Python.

*   `google-generativeai`: The official library from Google for interacting with the Gemini family of models.
*   `requests`: A simple yet powerful library for making HTTP requests, which is how we will communicate with the GitHub API.

### Documentation
*   **Google AI Python SDK:** [https://ai.google.dev/gemini-api/docs/libraries](https://ai.google.dev/gemini-api/docs/libraries)
*   **Requests Library:** [https://requests.readthedocs.io/en/latest/](https://requests.readthedocs.io/en/latest/)


### Code
```python
# The '!' lets us run a command-line command from within a Colab notebook.
# The '-q' flag stands for "quiet," reducing the amount of installation text.
# The '-U' flag ensures we install or upgrade to the latest version.
```

In [ ]:
!pip install --upgrade -q google-genai
!pip install -q requests

In [ ]:
import requests
import json

from google import genai
from google.genai import types
from google.colab import userdata


---

## Step 2: Securely Configuring the API Key

### Theory
An API Key is a secret token that you use to authenticate your requests with a service like Google's AI platform. **It is critical to keep your API keys private.** Hardcoding them directly into your script is a major security risk. Google Colab provides a "Secrets" manager, which is a secure way to store and access sensitive information like API keys without exposing them in your code.

![Secret Key in Colab](https://github.com/RashikaKarki/Workshop---Agent-and-Tool-Calling/blob/main/Images/colab_secret.png?raw=true)



In [ ]:
try:
    # userdata.get() securely retrieves the secret you stored in the Colab UI.
    api_key = userdata.get(<Name of your key>)
except userdata.SecretNotFoundError:
    print('ERROR: GOOGLE_API_KEY not found.')
    print('Please go to the "Key" icon on the left sidebar of Colab, add a new secret named GOOGLE_API_KEY, and paste your key in the value field.\n')
except Exception as e:
    print(f"An error occurred during API configuration: {e}\n")

This is the heart of our agent's ability to "act." We define a standard Python function that performs a specific task: fetching data from GitHub. The magic happens in the function's documentation string (the text between triple quotes `"""..."""`). The Gemini model will read this description, the function name, and its parameters to understand what this tool does and when to use it.

### Documentation
*   **GitHub REST API - Get a User:** [https://docs.github.com/en/rest/users/users?apiVersion=2022-11-28#get-a-user](https://docs.github.com/en/rest/users/users?apiVersion=2022-11-28#get-a-user)



In [ ]:
get_github_user_info_declaration = {
    "name": <name of function>,
    "description": <what does the function do>,
    "parameters": {
        "type": "object",
        "properties": {
            "username": {
                "type": <type>,
                "description": <description>,
            },
        },
        "required": <list of required fields>,
    },
}


In [ ]:
def get_github_user_info(username: str) -> str:
    """
    Fetches public information for a given GitHub username. The model will read this
    description to understand what this function does.

    Args:
        username (str): The GitHub username to look up. The model learns this is the
                        required input.

    Returns:
        A JSON string containing the user's public GitHub information or an error message.
    """
    print(f"--- TOOL ACTIVATED: Calling GitHub API for user: {username} ---")

    # This is the specific "endpoint" from the GitHub API menu to get user data.
    api_url = f"https://api.github.com/users/{username}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"--- GitHub API Error: {http_err} ---")
        return {"error": f"HTTP error occurred: {http_err}", "status_code": response.status_code}
    except Exception as e:
        print(f"--- An unexpected error occurred: {e} ---")
        return {"error": f"An unexpected error occurred: {e}"}

In [ ]:
# Call the function and see if it works as expected

### Initialize Gemini Model

Now we initialize the Gemini model. The most important part of this step is passing our `get_github_user_info` function into the `tools` parameter. This "registers" the tool with the model, making it a part of the agent's potential actions. The model now knows it has this special capability.

*   **Gemini API Tool Calling:** [https://ai.google.dev/docs/function_calling](https://ai.google.dev/docs/function_calling)



In [ ]:
client = genai.Client(api_key=<api key>)
tools = types.Tool(function_declarations= <add tools declaration here>)
config = types.GenerateContentConfig(tools=[tools])

In [ ]:
github_username = input("Enter a GitHub username to get information about (e.g., 'google', 'microsoft'): ")
user_prompt = f"Please provide a summary of the GitHub user {github_username}. Tell me their display name, their bio, how many public repos they have, and how many followers they have."


In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=user_prompt,
    config=config,
)

In [ ]:
# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")

    # call the function

    # Make another call by passion user data fetched by the tool
else:
    print("No function call found in the response.")
    print(response.text)
